In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
df = pd.read_csv("train[1].csv")

print(df.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [ ]:
#Binary classifcaiton! Toxic vs. Nontoxic
df['toxic'] = df['toxic'].astype(int)

#Feature and target
X = df['comment_text']
y = df['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
#TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.fit_transform(X_test)

In [ ]:
#Train
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_test_tfidf)
print("Accurary:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accurary: 0.8999530001566661
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     28856
           1       0.17      0.01      0.02      3059

    accuracy                           0.90     31915
   macro avg       0.54      0.50      0.48     31915
weighted avg       0.83      0.90      0.86     31915



# Multi-Label Classifcation

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

X = df['comment_text']
y = df[label_cols]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#Vectorization
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.fit_transform(X_test)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

classifier = OneVsRestClassifier(LogisticRegression(max_iter=1000))
#Train
classifier.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [ ]:
y_pred = classifier.predict(X_test_tfidf)

for i, col in enumerate(label_cols):
  print(f"Label: {col}")
  print(f"Accuracy: {accuracy_score(y_test[col], y_pred[:, i])}")
  print(classification_report(y_test[col], y_pred[:,i]))

Label: toxic
Accuracy: 0.8993263355788814
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     28859
           1       0.09      0.01      0.01      3056

    accuracy                           0.90     31915
   macro avg       0.50      0.50      0.48     31915
weighted avg       0.83      0.90      0.86     31915

Label: severe_toxic
Accuracy: 0.989942033526555
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     31594
           1       0.00      0.00      0.00       321

    accuracy                           0.99     31915
   macro avg       0.49      0.50      0.50     31915
weighted avg       0.98      0.99      0.98     31915

Label: obscene
Accuracy: 0.945135516214946
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     30200
           1       0.11      0.00      0.01      1715

    accuracy                           0.95     3

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Deep Learning

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


df = df.dropna()

# Labels
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
X = df['comment_text']
y = df[label_cols]

# Text preprocessing
max_words = 50000
max_len = 150

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(6, activation='sigmoid')  # 6 labels
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred, target_names=label_cols))

subset_accuracy = np.mean(np.all(y_pred == y_test.values, axis=1))
print(f"Subset accuracy (exact match across all 6 labels): {subset_accuracy:.4f}")

# Average accuracy per label
per_label_accuracy = (y_pred == y_test.values).mean(axis=0)
for label, acc in zip(label_cols, per_label_accuracy):
    print(f"Accuracy for {label}: {acc:.4f}")

# Mean of per-label accuracies
mean_accuracy = per_label_accuracy.mean()
print(f"Mean per-label accuracy: {mean_accuracy:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 35s 35ms/step - accuracy: 0.9018 - loss: 0.1538 - val_accuracy: 0.9940 - val_loss: 0.0547
Epoch 2/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.9940 - loss: 0.0493 - val_accuracy: 0.9940 - val_loss: 0.0510
Epoch 3/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9939 - loss: 0.0421 - val_accuracy: 0.9940 - val_loss: 0.0515
Epoch 4/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.9930 - loss: 0.0376 - val_accuracy: 0.9939 - val_loss: 0.0535
Epoch 5/5
898/898 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9920 - loss: 0.0338 - val_accuracy: 0.9937 - val_loss: 0.0584
998/998 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step
               precision    recall  f1-score   support

        toxic       0.83      0.75      0.79      3056
 severe_toxic       0.47      0.35      0.40       321
      obscene       0.84      0.77      0.80      1715
       threat       0.00      0.00      0.00        74
       insult       0.73      0.65    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_